In [1]:
from bs4 import BeautifulSoup
import re
import requests
import string

In [ ]:
JEOP_URL = "http://www.j-archive.com/listseasons.php"
Q_PATH = "qs.csv"
COLUMNS = "QUESTION,ANSWER,SEASON,DOLLARS"

In [2]:
class Clue(object):
    """Class to hold, clean, and write data from a single Jeopardy clue."""
    
    def __init__(self, ssn, raw_html):
        self._raw_html = raw_html
        self._ssn = ssn
        self._q = None
        self._dollars = None
        self._a = None
        # self._date = None
        # self._ep = None
        
    def parse(self):
        """Extract data from the raw html."""
        self._q = self._raw_html.find(class_="clue_text").text
        self._dollars = self._raw_html.find(class_="clue_value").text
        self._a = str(clue).split("correct_response")[1].split("<table")[0]
    
    def _clean_text(self, s):
        """Remove punctuation from a string and convert to lowercase."""
        s = s.replace(string.punctuation, "")
        s = s.to_lower()
        return s        
    
    def to_string(self):
        """Create write-able string."""
        q = self._clean_text(self._q)
        a = self._clean_text(self._a)
        s = "{},{},{},{}".format(q, a, self._ssn, self._dollars)
        return s

In [9]:
class Game(object):
    """Class to hold and parse data from a single Jeopardy game."""
    
    def __init__(self, url, ssn):
        self._url = url
        self._ssn = ssn
        self._game_id = url.split("=")[1]
        self._page = requests.get(url)
        self._parsed_page = BeautifulSoup(self._page.content, "lxml")
        
    def get_clues(self, round_name):
        """Method to return all of the data for a given round.
        
        Args:
          round_name: (string) one of jeopardy, double_jeopardy, final_jeopardy.
        
        Returns:
          list of Clue objects.
        """
        assert round_name in ["jeopardy", "double_jeopardy", "final_jeopardy"]
        
        table = self._parsed_page.find_all(id=round_name + "_round")
        assert(len(table) == 1)
        
        clues = []
        raw_clues = table[0].find_all(class_="clue")
        if round_name == "final_jeopardy":
            assert(len(raw_clues) == 1)
        else:
            assert(len(raw_clues) == 30)
            
        for raw_clue in raw_clues:
            c = Clue(raw_clue)
            c.parse()
            clues.append(c)
            
        return clues

In [4]:
class Season(object):
    """Class to parse data for a single Jeopardy season."""
    
    def __init__(self, url):
        self._url = url
        self.ssn = url.split("=")[1]
        self._page = requests.get(url)
        self._parsed_page = BeautifulSoup(self._page.content, "lxml")
        
    def get_links(self):
        """Returns the links to all of the games on the season page."""
        elements = self._parsed_page.find_all("a")
        all_links = [e.get("href") for e in elements]
        links = [l for l in all_links if "game_id" in l]
        return links

In [10]:
class Jeopardy(object):
    """Class to scrape all Jeopardy data."""
    
    def __init__(self):
        self._url = JEOP_URL
        self._page = requests.get(self._url)
        self._parsed_page = BeautifulSoup(self._page.content, "lxml")
        self._season_links = []
        self._clues = []
        self._last_saved = 0
        
    def get_season_links(self):
        """Grabs the links to all of the seasons on the main page."""
        elements = self._parsed_page.find_all("a")
        all_links = [e.get("href") for e in elements]
        links = [l for l in all_links if "showseason" in l]
        seen = set()
        formatted_links = []
        for l in links:
            ssn = l.split("=")[1]
            if ssn in seen:
                continue
            if l.startswith("http"):
                formatted_links.append(l)
            else:
                formatted_links.append("http://www.j-archive.com/{}".format(l))
            seen.add(ssn)
        self._season_links = formatted_links
        
    def _save(self):
        """Dumps existing clue data to a CSV."""
        if self._last_saved == 0:
            with open(Q_PATH, "wb") as f:
                f.write(COLUMNS)
        clue_strings = [c.to_string() for c in self._clues[self._last_saved:]]
        self._last_saved += len(clue_strings)
        print("Saving {} clues, for a total of {} clues saved.".format(len(clue_strings), self._last_saved)
        with open(Q_PATH, "wba") as f:
            f.write("\n".join(clue_strings))
        
    def get_clues(self):
        """Grabs data about each clue, utilizing the Season, Game, and Clue objects."""
        assert len(self._season_links) != 0
        
        for i, season_link in enumerate(self._season_links):
            
            s = Season(season_link)
            game_links = s.get_links()
            ssn = s.ssn
            del s
            
            print("Parsing games for season {}.".format(ssn))
            for game_link in game_links:
                g = Game(game_link, ssn)
                clues = g.get_clues("jeopardy")
                self._clues.extend(clues)
                self._save()
                del g
    
j = Jeopardy()
j.get_season_links()
j.get_clues()

Initialized Jeopardy object.
Finished getting seasons.



<td class="clue">
<table>
<tr>
<td>
<div onclick="togglestick('clue_J_1_1_stuck')" onmouseout="toggle('clue_J_1_1', 'clue_J_1_1_stuck', 'The Turtle Creek this establishment in northern Michigan is owned by the Grand Traverse Band of Ottawa &amp; Chippewa Indians')" onmouseover="toggle('clue_J_1_1', 'clue_J_1_1_stuck', '&lt;em class=&quot;correct_response&quot;&gt;a casino&lt;/em&gt;&lt;br /&gt;&lt;br /&gt;&lt;table width=&quot;100%&quot;&gt;&lt;tr&gt;&lt;td class=&quot;right&quot;&gt;Lindsay&lt;/td&gt;&lt;/tr&gt;&lt;/table&gt;')">
<table class="clue_header">
<tr>
<td class="clue_unstuck" id="clue_J_1_1_stuck">   </td>
<td class="clue_value">$200</td>
<td class="clue_order_number"><a href="suggestcorrection.php?clue_id=378768" rel="nofollow" title="Suggest a correction for this clue">23</a></td>
</tr>
</table>
</div>
</td>
</tr>
<tr>
<td class="clue_text" id="clue_J_1_1">The Turtle Creek this establishment in northern Michigan is




<td class="clue">
<table>
<tr>
<td>
<div onclick="togglestick('clue_J_1_1_stuck')" onmouseout="toggle('clue_J_1_1', 'clue_J_1_1_stuck', 'As a symbol of American success, &lt;a href=&quot;http://www.j-archive.com/media/2020-01-16_J_04.jpg&quot; target=&quot;_blank&quot;&gt;it&lt;/a&gt; dates back to the Colonial Era')" onmouseover="toggle('clue_J_1_1', 'clue_J_1_1_stuck', '&lt;em class=&quot;correct_response&quot;&gt;a picket fence&lt;/em&gt;&lt;br /&gt;&lt;br /&gt;&lt;table width=&quot;100%&quot;&gt;&lt;tr&gt;&lt;td class=&quot;right&quot;&gt;Veronica&lt;/td&gt;&lt;/tr&gt;&lt;/table&gt;')">
<table class="clue_header">
<tr>
<td class="clue_unstuck" id="clue_J_1_1_stuck">   </td>
<td class="clue_value">$200</td>
<td class="clue_order_number"><a href="suggestcorrection.php?clue_id=378692" rel="nofollow" title="Suggest a correction for this clue">4</a></td>
</tr>
</table>
</div>
</td>
</tr>
<tr>
<td class="clue_text" id="clue_J_1_1">As a symbol of American success, <a href="http://www.j




<td class="clue">
<table>
<tr>
<td>
<div onclick="togglestick('clue_J_1_1_stuck')" onmouseout="toggle('clue_J_1_1', 'clue_J_1_1_stuck', 'Her grandson said she suggested the use of a 5-pointed star on the flag vs. George Washington\'s chosen 6-pointer')" onmouseover="toggle('clue_J_1_1', 'clue_J_1_1_stuck', '&lt;em class=&quot;correct_response&quot;&gt;Betsy Ross&lt;/em&gt;&lt;br /&gt;&lt;br /&gt;&lt;table width=&quot;100%&quot;&gt;&lt;tr&gt;&lt;td class=&quot;right&quot;&gt;Laura&lt;/td&gt;&lt;/tr&gt;&lt;/table&gt;')">
<table class="clue_header">
<tr>
<td class="clue_unstuck" id="clue_J_1_1_stuck">   </td>
<td class="clue_value">$200</td>
<td class="clue_order_number"><a href="suggestcorrection.php?clue_id=378633" rel="nofollow" title="Suggest a correction for this clue">1</a></td>
</tr>
</table>
</div>
</td>
</tr>
<tr>
<td class="clue_text" id="clue_J_1_1">Her grandson said she suggested the use of a 5-pointed star on the flag vs. George Washington's chosen 6-pointer</td>
</tr>
</t




<td class="clue">
<table>
<tr>
<td>
<div onclick="togglestick('clue_J_1_1_stuck')" onmouseout="toggle('clue_J_1_1', 'clue_J_1_1_stuck', 'Had Rum, North Carolina')" onmouseover="toggle('clue_J_1_1', 'clue_J_1_1_stuck', '&lt;em class=&quot;correct_response&quot;&gt;Durham [HAD RUM]&lt;/em&gt;&lt;br /&gt;&lt;br /&gt;&lt;table width=&quot;100%&quot;&gt;&lt;tr&gt;&lt;td class=&quot;right&quot;&gt;Alissa&lt;/td&gt;&lt;/tr&gt;&lt;/table&gt;')">
<table class="clue_header">
<tr>
<td class="clue_unstuck" id="clue_J_1_1_stuck">   </td>
<td class="clue_value">$200</td>
<td class="clue_order_number"><a href="suggestcorrection.php?clue_id=378454" rel="nofollow" title="Suggest a correction for this clue">1</a></td>
</tr>
</table>
</div>
</td>
</tr>
<tr>
<td class="clue_text" id="clue_J_1_1">Had Rum, North Carolina</td>
</tr>
</table>
</td>



<td class="clue">
<table>
<tr>
<td>
<div onclick="togglestick('clue_J_2_1_stuck')" onmouseout="toggle('clue_J_2_1', 'clue_J_2_1_stuck', 'Giving new meaning t




<td class="clue">
<table>
<tr>
<td>
<div onclick="togglestick('clue_J_1_1_stuck')" onmouseout="toggle('clue_J_1_1', 'clue_J_1_1_stuck', '&lt;a href=&quot;http://www.j-archive.com/media/2020-01-13_J_15.jpg&quot; target=&quot;_blank&quot;&gt;It\'s&lt;/a&gt; roughly a third the size of the contiguous United States')" onmouseover="toggle('clue_J_1_1', 'clue_J_1_1_stuck', '&lt;em class=&quot;correct_response&quot;&gt;Argentina&lt;/em&gt;&lt;br /&gt;&lt;br /&gt;&lt;table width=&quot;100%&quot;&gt;&lt;tr&gt;&lt;td class=&quot;right&quot;&gt;Daniel&lt;/td&gt;&lt;/tr&gt;&lt;/table&gt;')">
<table class="clue_header">
<tr>
<td class="clue_unstuck" id="clue_J_1_1_stuck">   </td>
<td class="clue_value">$200</td>
<td class="clue_order_number"><a href="suggestcorrection.php?clue_id=378402" rel="nofollow" title="Suggest a correction for this clue">15</a></td>
</tr>
</table>
</div>
</td>
</tr>
<tr>
<td class="clue_text" id="clue_J_1_1"><a href="http://www.j-archive.com/media/2020-01-13_J_15.jpg" tar

...




<td class="clue">
<table>
<tr>
<td>
<div onclick="togglestick('clue_J_1_1_stuck')" onmouseout="toggle('clue_J_1_1', 'clue_J_1_1_stuck', 'Peter Pan can bring visitors to this island, home to the Lost Boys')" onmouseover="toggle('clue_J_1_1', 'clue_J_1_1_stuck', '&lt;em class=&quot;correct_response&quot;&gt;Never-Never Land&lt;/em&gt;&lt;br /&gt;&lt;br /&gt;&lt;table width=&quot;100%&quot;&gt;&lt;tr&gt;&lt;td class=&quot;right&quot;&gt;Jason&lt;/td&gt;&lt;/tr&gt;&lt;/table&gt;')">
<table class="clue_header">
<tr>
<td class="clue_unstuck" id="clue_J_1_1_stuck">   </td>
<td class="clue_value">$200</td>
<td class="clue_order_number"><a href="suggestcorrection.php?clue_id=371480" rel="nofollow" title="Suggest a correction for this clue">27</a></td>
</tr>
</table>
</div>
</td>
</tr>
<tr>
<td class="clue_text" id="clue_J_1_1">Peter Pan can bring visitors to this island, home to the Lost Boys</td>
</tr>
</table>
</td>



<td class="clue">
<table>
<tr>
<td>
<div onclick="togglestick('clue_J_2




<td class="clue">
<table>
<tr>
<td>
<div onclick="togglestick('clue_J_1_1_stuck')" onmouseout="toggle('clue_J_1_1', 'clue_J_1_1_stuck', 'September\'s birthstone is &lt;a href=&quot;http://www.j-archive.com/media/2019-09-12_J_28.jpg&quot; target=&quot;_blank&quot;&gt;this&lt;/a&gt; precious gem that\'s a variety of corundum')" onmouseover="toggle('clue_J_1_1', 'clue_J_1_1_stuck', '&lt;em class=&quot;correct_response&quot;&gt;sapphire&lt;/em&gt;&lt;br /&gt;&lt;br /&gt;&lt;table width=&quot;100%&quot;&gt;&lt;tr&gt;&lt;td class=&quot;right&quot;&gt;Jason&lt;/td&gt;&lt;/tr&gt;&lt;/table&gt;')">
<table class="clue_header">
<tr>
<td class="clue_unstuck" id="clue_J_1_1_stuck">   </td>
<td class="clue_value">$200</td>
<td class="clue_order_number"><a href="suggestcorrection.php?clue_id=371709" rel="nofollow" title="Suggest a correction for this clue">28</a></td>
</tr>
</table>
</div>
</td>
</tr>
<tr>
<td class="clue_text" id="clue_J_1_1">September's birthstone is <a href="http://www.j-archiv




<td class="clue">
</td>



<td class="clue">
<table>
<tr>
<td>
<div onclick="togglestick('clue_J_2_1_stuck')" onmouseout="toggle('clue_J_2_1', 'clue_J_2_1_stuck', '&quot;Don\'t touch that&quot; this, the tuning knob on a radio or television')" onmouseover="toggle('clue_J_2_1', 'clue_J_2_1_stuck', '&lt;em class=&quot;correct_response&quot;&gt;dial&lt;/em&gt;&lt;br /&gt;&lt;br /&gt;&lt;table width=&quot;100%&quot;&gt;&lt;tr&gt;&lt;td class=&quot;right&quot;&gt;Jason&lt;/td&gt;&lt;/tr&gt;&lt;/table&gt;')">
<table class="clue_header">
<tr>
<td class="clue_unstuck" id="clue_J_2_1_stuck">   </td>
<td class="clue_value">$200</td>
<td class="clue_order_number"><a href="suggestcorrection.php?clue_id=371656" rel="nofollow" title="Suggest a correction for this clue">20</a></td>
</tr>
</table>
</div>
</td>
</tr>
<tr>
<td class="clue_text" id="clue_J_2_1">"Don't touch that" this, the tuning knob on a radio or television</td>
</tr>
</table>
</td>



<td class="clue">
</td>



<td class="clue">
<t




<td class="clue">
</td>



<td class="clue">
<table>
<tr>
<td>
<div onclick="togglestick('clue_J_2_1_stuck')" onmouseout="toggle('clue_J_2_1', 'clue_J_2_1_stuck', 'Hey there, fancy (above the) pants! The name of this tuxedo staple comes from the Hindi for a waist band')" onmouseover="toggle('clue_J_2_1', 'clue_J_2_1_stuck', '&lt;em class=&quot;correct_response&quot;&gt;a cummerbund&lt;/em&gt;&lt;br /&gt;&lt;br /&gt;&lt;table width=&quot;100%&quot;&gt;&lt;tr&gt;&lt;td class=&quot;right&quot;&gt;Valerie&lt;/td&gt;&lt;/tr&gt;&lt;/table&gt;')">
<table class="clue_header">
<tr>
<td class="clue_unstuck" id="clue_J_2_1_stuck">   </td>
<td class="clue_value">$200</td>
<td class="clue_order_number"><a href="suggestcorrection.php?clue_id=371413" rel="nofollow" title="Suggest a correction for this clue">6</a></td>
</tr>
</table>
</div>
</td>
</tr>
<tr>
<td class="clue_text" id="clue_J_2_1">Hey there, fancy (above the) pants! The name of this tuxedo staple comes from the Hindi for a waist band<




<td class="clue">
<table>
<tr>
<td>
<div onclick="togglestick('clue_J_1_1_stuck')" onmouseout="toggle('clue_J_1_1', 'clue_J_1_1_stuck', '2004:&lt;br /&gt;This pioneering Toyota model')" onmouseover="toggle('clue_J_1_1', 'clue_J_1_1_stuck', '(Victor: What is Corolla?)&lt;br /&gt;...&lt;br /&gt;[The end-of-round signal sounds.]&lt;br /&gt;&lt;br /&gt;&lt;em class=&quot;correct_response&quot;&gt;the Prius&lt;/em&gt;&lt;br /&gt;&lt;br /&gt;&lt;table width=&quot;100%&quot;&gt;&lt;tr&gt;&lt;td class=&quot;wrong&quot;&gt;Victor&lt;/td&gt;&lt;/tr&gt;&lt;/table&gt;&lt;table width=&quot;100%&quot;&gt;&lt;tr&gt;&lt;td class=&quot;wrong&quot;&gt;Triple Stumper&lt;/td&gt;&lt;/tr&gt;&lt;/table&gt;')">
<table class="clue_header">
<tr>
<td class="clue_unstuck" id="clue_J_1_1_stuck">   </td>
<td class="clue_value">$200</td>
<td class="clue_order_number"><a href="suggestcorrection.php?clue_id=371368" rel="nofollow" title="Suggest a correction for this clue">29</a></td>
</tr>
</table>
</div>
</td>
</t